In [0]:
%fs ls /Volumes/pocameet_dev_catalog/pocameet_silver/pocameetdatafiles/SalesData/


In [0]:
df = spark.read.format("csv").option(
    "header",
    "true"
).load(
    "/Volumes/pocameet_dev_catalog/pocameetdata_bronze/pocameetdatafiles/IncrementalSales.csv"
)
display(df)

In [0]:
from pyspark.sql import functions as F
df = df.withColumn('model_category',F.split(df['Model_ID'],'-')[0])
display(df)

In [0]:
from pyspark.sql.functions import date_format
df = df.withColumn('revenue_new',df['Revenue'].cast('double'))
df = df.withColumn('units_new',df['Units_Sold'].cast('double'))
df = df.withColumn('SysTime',date_format(F.current_timestamp(),"yyyy-MM-dd HH:mm:ss"))
df = df.withColumn('UserID',F.current_user())

In [0]:
from pyspark.sql.functions import col,lpad,to_date,concat_ws
df = df.withColumn('revenue_per_unit',df['revenue_new']/df['units_new'])
df = df.withColumn('combined_date',concat_ws("-",col("Year"),lpad(col("Month"),2,"0"),lpad(col("Day"),2,"0")))


In [0]:
display(df)

In [0]:
from pyspark.sql.functions import sum as F_sum

df.groupBy('Year','BranchName').agg(
    F_sum('units_Sold').alias('total_units_sold')
).sort('Year','total_units_sold',ascending=[True,False]).display()

Databricks visualization. Run in Databricks to view.

In [0]:
df.write.format('parquet') \
    .mode('append') \
    .option('path','/Volumes/pocameet_dev_catalog/pocameet_silver/pocameetdatafiles/SalesData') \
    .save()

In [0]:
%sql
select * from PARQUET.`/Volumes/pocameet_dev_catalog/pocameet_silver/pocameetdatafiles/SalesData`